### TensorBoard. Визуализация графов. Изменение скорости обучения. 

In [11]:
import tensorflow as tf
from packaging import version
from datetime import datetime
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from keras import datasets, layers, models

Импортируем Tensorboard и определим версию плагина, затем очистим директории от логов предыдущих сессий (если таковые имеются)

In [13]:
import tensorboard
tensorboard.__version__

'2.9.1'

In [14]:
!rm -rf. /logs/

rm: illegal option -- .
usage: rm [-f | -i] [-dPRrvW] file ...
       unlink file


Теперь описываем модель keras для обучения. Сделаем слои плоскими.

In [15]:
model = Sequential([
    keras.layers.Flatten(input_shape = (28, 28)),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation = 'softmax')
])

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'])

Здесь мы задали размер изображений 28 х 28, затем добавили полносвязный слой размерности 32. Далее задали выброс чтобы исключить переобучение (эффект, при котором точность валидации ниже точности обучения, возникает когда данных слишком много). Выделим 20% данных **keras.layers.Dropout(0.2)**. Добавим второй полносвязный слой и используем метод активации **'softmax'**.

Типов одежды у нас 10. Нейросеть будет работать как фильтр - принимать изображение 28 х 28 и делать вывод, к какой категории относится то или иное изображение.

Теперь разделим датасет на тренировочную и валидационную часть с помощью множественного присваивания. Нормализуем данные разделив значения на 255. Поскольку мы используем изображения, то описываются они как последовательность пикселей в цветовой модели RGB - каждый пиксель описывается в три числа в диапазоне от 0 до 255, потому и делим все на 255, чтобы привести к диапазону от 0 до 1. Это называется нормализация. С таким форматом данных нейросеть далее и будет работать.

In [16]:
(train_images, train_labels), _= keras.datasets.fashion_mnist.load_data()

train_images = train_images / 255.0

Теперь задаем директорую для хранения логов мини-отчетов обучения. Используем

In [19]:
logdir = 'logs/fit' + datetime.now().strftime('%Y%m%d-%H%m%S')
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(
    train_images,
    train_labels,
    batch_size = 64,
    epochs = 5,
    callbacks = [tensorboard_callback]
)


Epoch 1/5
938/938 [==============================] - 2s 2ms/step - loss: 0.7072 - accuracy: 0.7549
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4903 - accuracy: 0.8269
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4489 - accuracy: 0.8407: 1s -
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4251 - accuracy: 0.8486
Epoch 5/5
938/938 [==============================] - 2s 3ms/step - loss: 0.4121 - accuracy: 0.8516: 0s - loss: 0.4114 - accura


In [22]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [25]:
@tf.function
def my_func(x,y):
    return tf.nn.relu(tf.matmul(x,y))

stamp = datetime.now().strftime('%Y%m%d-%H%m%S')
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

x = tf.random.uniform((3,3))
y = tf.random.uniform((3,3))

In [26]:
tf.summary.trace_on(graph = True, profiler = True)
z = my_func(x,y)

with writer.as_default():
    tf.summary.trace_export(
        name = 'my_func_trace',
        step = 0,
        profiler_outdir = logdir
    )

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


2022-07-18 22:39:43.766832: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-07-18 22:39:43.767906: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-07-18 22:39:43.931809: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-07-18 22:39:43.949343: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.


In [27]:
%tensorboard --logdir logs/func

UsageError: Line magic function `%tensorboard` not found.
